## Calculating the GTO but with regularization terms

In [1]:
## Imports
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from utils import *
import xarray as xr
import time

### Simple linear model with lasso (l1) regularization

This can be done using the built in scikit-learn Lasso class.

In [56]:
delta_SST = xr.open_dataarray("delta_SST_pr.nc").compute()
delta_precip = xr.open_dataarray("delta_precip_10deg.nc").compute()

In [57]:
delta_SST = delta_SST.sel(lat=slice(60, -60))

Let us compute the GTO for the point with lat = 25.0 and lon = 5.0. To do this we extract the target vector, the precipitation anomaly $\vec{y}$ as

In [58]:
mylat = 25
mylon = 5
y = point_delta_precip_vector(delta_precip, mylat=mylat, mylon=mylon, time_avg='ANNUAL', weight_by_lat=True)

In [59]:
print(y.shape)

(5545,)


The matrix $\mathbf{X}$ represents the SST field perturbations and has dimensions (number of training examples = 5545) x (number of gridpoints = 18624). We must therefore flatten the delta_SST array to these dimensions.

In [60]:
delta_SST.data.shape

(5545, 97, 192)

In [61]:
X = delta_SST.data.reshape(5545, -1)

In [62]:
X.shape

(5545, 18624)

Scale the $\mathbf{X}$ data first using StandardScaler. This removes the mean and scales to unit variance (calculates the z-scores).

In [41]:
#scaler = StandardScaler().fit(X)
#X = scaler.transform(X)

In [67]:
# test method first for a small number of gridpoints
X_testing = X[:,:500]
X_testing.shape

(5545, 500)

### Simple linear regression, no regularization

In [73]:
t0 = time.time()
coeffs = np.array(
for X_gridpoint in X.T: # need to loop over individual gridpoints
    reg = LinearRegression().fit(X_gridpoint.reshape(-1, 1), y)
    coeffs.append(reg.coef_[0])
t1 = time.time()
print("Time elapsed: {} seconds".format(t1 - t0))

Time elapsed: 10.408801555633545 seconds


Q: difference between doing individual regressions for each gridpoint, or doing multiple linear regression (much much slower!) Lasso, custom loss function etc. seems to suffer from the same problem.